In [1]:
# Ahmet SOYYİĞİT
# The code is working but performance is not so good,
# please inform me if you have any suggestions
# lstm inputs are 3D, [sequence,batch_num,data]
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import dataUtils as du
import numpy as np

torch.set_default_tensor_type('torch.cuda.FloatTensor')
torch.manual_seed(1)

print(torch.cuda.is_available())
print(torch.backends.cudnn.version())

True
7003


In [2]:
# [timesteps, numOfSequences(number of batch elements), inputSize]
trainDir = "./MSRAction3DSkeletonReal3D"
crossSubject = False
if crossSubject:
    trainData, trainLabels, testData, testLabels = du.readMSRDatasetCrossSubject(trainDir,40,10)
    trainData = du.organizeSkeletonData(trainData)
    testData = du.organizeSkeletonData(testData)
else:
    trainData, trainLabels = du.readMSRDataset(trainDir,40)
    tLEn = trainData.shape[1]
    p = np.random.permutation(trainData.shape[1])
    trainData, trainLabels = trainData[:,p,:],trainLabels[p]
    testData, testLabels = trainData[:,(tLEn-tLEn//5):,:],trainLabels[(tLEn-tLEn//5):]
    trainData, trainLabels = trainData[:,:(tLEn-tLEn//5),:],trainLabels[:(tLEn-tLEn//5)]
print(trainData.shape)
print(testData.shape)
print(trainLabels.shape)
print(testLabels.shape)

Loading MSR 3D Data, data directory ./MSRAction3DSkeletonReal3D
(40, 18329, 60)
(40, 4582, 60)
(18329,)
(4582,)


In [3]:
class denseLSTM(nn.Module):

    def __init__(self):
        super(denseLSTM, self).__init__()
        
        #self.fc_init_layer = nn.Linear(60,12, bias=False)
        self.l1_LSTM  = nn.LSTM(60,64)
        #There is a dropout here
        self.l2_LSTM  = nn.LSTM(60+64,128)
        #There is a dropout here
        self.l3_LSTM  = nn.LSTM(60+64+128,256)
        #There is a dropout here
        self.fc_last_layer  = nn.Linear(256,20)

    def init_hidden(self,batchSize):
        self.hidden = [
            (
                autograd.Variable(torch.zeros(1, batchSize, hidden_dim).cuda()),
                autograd.Variable(torch.zeros(1, batchSize, hidden_dim).cuda())
            ) for hidden_dim in [64,128,256]
        ]
    
    def forward(self, inpBatch):
        self.init_hidden(inpBatch.size()[1])
        
        # Forward input through network        
        l1_lstm_out, self.hidden[0] = self.l1_LSTM(
                                                    inpBatch,
                                                    self.hidden[0]
                                                   )        
        
        l2_lstm_out, self.hidden[1] = self.l2_LSTM(
                                                    F.dropout(torch.cat( (inpBatch,l1_lstm_out), 2 ),0.95),
                                                    self.hidden[1]
                                                   )        
        
        l3_lstm_out, self.hidden[2] = self.l3_LSTM(
                                                    F.dropout(torch.cat((inpBatch,l1_lstm_out,l2_lstm_out),2),0.95),
                                                    self.hidden[2]
                                                   )
        

        
        fc_linear_out =self.fc_last_layer(F.dropout(l3_lstm_out[-1],0.95))
        return fc_linear_out

In [4]:
#create network object
model = denseLSTM().cuda()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)
torch.cuda.empty_cache()

In [5]:
def dataBatchLens(inputData,batchSize):
    l = inputData.shape[1]
    bLens = [ [b,b+batchSize] for b in range(0,l-l%batchSize,batchSize) ]
    if l-l%batchSize != l:
        bLens.append([l-l%batchSize, l])
    return bLens
trainBatchLens = dataBatchLens(trainData,64) 
testBatchLens = dataBatchLens(testData,64) 
#print(trainBatchLens)
print(testBatchLens)

[[0, 64], [64, 128], [128, 192], [192, 256], [256, 320], [320, 384], [384, 448], [448, 512], [512, 576], [576, 640], [640, 704], [704, 768], [768, 832], [832, 896], [896, 960], [960, 1024], [1024, 1088], [1088, 1152], [1152, 1216], [1216, 1280], [1280, 1344], [1344, 1408], [1408, 1472], [1472, 1536], [1536, 1600], [1600, 1664], [1664, 1728], [1728, 1792], [1792, 1856], [1856, 1920], [1920, 1984], [1984, 2048], [2048, 2112], [2112, 2176], [2176, 2240], [2240, 2304], [2304, 2368], [2368, 2432], [2432, 2496], [2496, 2560], [2560, 2624], [2624, 2688], [2688, 2752], [2752, 2816], [2816, 2880], [2880, 2944], [2944, 3008], [3008, 3072], [3072, 3136], [3136, 3200], [3200, 3264], [3264, 3328], [3328, 3392], [3392, 3456], [3456, 3520], [3520, 3584], [3584, 3648], [3648, 3712], [3712, 3776], [3776, 3840], [3840, 3904], [3904, 3968], [3968, 4032], [4032, 4096], [4096, 4160], [4160, 4224], [4224, 4288], [4288, 4352], [4352, 4416], [4416, 4480], [4480, 4544], [4544, 4582]]


In [6]:
for epoch in range(100): 
    correctPredictionsTr = 0
    totalLossTr = 0.0
    #Shuffle data before each epoch
    p = np.random.permutation(trainData.shape[1])
    trainData, trainLabels = trainData[:,p,:],trainLabels[p]
    for bLen in trainBatchLens:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Variables of word indices.
        batch = autograd.Variable(torch.from_numpy(trainData[:,bLen[0]:bLen[1]]).cuda())
        labels = autograd.Variable(torch.from_numpy(trainLabels[bLen[0]:bLen[1]]).cuda())

        # Step 3. Run our forward pass.
        results = model(batch)
        _ , idx = torch.max(torch.exp(results),1)
        correctPredictionsTr += torch.sum(torch.eq(idx,labels)).data[0]
        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(results, labels)
        totalLossTr += loss.data[0]
        loss.backward()
        optimizer.step()
    # After each epoch, test
    correctPredictionsTe = 0
    totalLossTe = 0.0
    for bLen in testBatchLens:
        batch = autograd.Variable(torch.from_numpy(testData[:,bLen[0]:bLen[1]]).cuda())
        labels = autograd.Variable(torch.from_numpy(testLabels[bLen[0]:bLen[1]]).cuda())
        results = model(batch)
        _ , idx = torch.max(torch.exp(results),1)
        correctPredictionsTe += torch.sum(torch.eq(idx,labels)).data[0]
        loss = loss_function(results,labels)
        totalLossTe += loss.data[0]
    print("Epoch " + str(epoch+1)+",Train loss    : " + str(totalLossTr / trainLabels.shape[0]))
    print("Epoch " + str(epoch+1)+",Test  loss    : " + str(totalLossTe / testLabels.shape[0]))
    print("Epoch " + str(epoch+1)+",Train accuracy: " + str(correctPredictionsTr) + "/" + 
          str(trainLabels.shape[0]) + " %" + str(int(correctPredictionsTr/trainLabels.shape[0]*100)))
    print("Epoch " + str(epoch+1)+",Test  accuracy: " + str(correctPredictionsTe) + "/" + 
          str(testLabels.shape[0]) + " %" + str(int(correctPredictionsTe/testLabels.shape[0]*100)))

#Non cross subject


# Cross-subject Epoch 50
# subject  1  2  3  4  5  6  7  8  9 10 TOTAL
# test    56 46 46 63 49 62 63 63 44 50 54.2
# train   92 92 92 92 92 92 92 92 92 92  92

Epoch 1,Train loss    : 0.04237367880637715
Epoch 1,Test  loss    : 0.11155219501618577
Epoch 1,Train accuracy: 2414/18329 %13
Epoch 1,Test  accuracy: 0/4582 %0
Epoch 2,Train loss    : 0.03643389835354544
Epoch 2,Test  loss    : 0.12422808509162293
Epoch 2,Train accuracy: 4245/18329 %23
Epoch 2,Test  accuracy: 0/4582 %0
Epoch 3,Train loss    : 0.032387243770682826
Epoch 3,Test  loss    : 0.12403951280245037
Epoch 3,Train accuracy: 5695/18329 %31
Epoch 3,Test  accuracy: 65/4582 %1
Epoch 4,Train loss    : 0.027810387840585264
Epoch 4,Test  loss    : 0.13094702967906405
Epoch 4,Train accuracy: 7412/18329 %40
Epoch 4,Test  accuracy: 176/4582 %3
Epoch 5,Train loss    : 0.025499170017330885
Epoch 5,Test  loss    : 0.12911445633494037
Epoch 5,Train accuracy: 8421/18329 %45
Epoch 5,Test  accuracy: 238/4582 %5


KeyboardInterrupt: 